Calculate accuracy, IoU, Dice of the results

In [7]:
import os
import skimage.io
import numpy as np
import matplotlib.pyplot as plt

def plot_two(img1, img2, name='image'):
    fig, ax = plt.subplots(1, 2, figsize=(15,7))
#     fig.suptitle(name)
    ax[0].imshow(img1, cmap='gray')
    ax[1].imshow(img2, cmap='gray')
    ax[0].set_xticks([])
    ax[1].set_xticks([])
    ax[0].set_yticks([])
    ax[1].set_yticks([])

    plt.tight_layout()


os.makedirs('data/FISH/otsu_results', exist_ok=True)

In [8]:
img_name = 22
# our predict results
msk1 = skimage.io.imread('data/FISH/pred_masks/{:04d}.png'.format(img_name))
b1 = np.zeros(msk1.shape)
b1[msk1 > 0] = 1

# matlab annotation
msk2 = skimage.io.imread('data/FISH/raw_masks/{:04d}.png'.format(img_name))
b2 = np.zeros(msk2.shape)
b2[msk2 > 0] = 1

#  double dip predict
msk3 = skimage.io.imread('output/FISH/final/{:04d}.png_mask.jpg'.format(img_name))
b3 = np.zeros(msk3.shape)
b3[msk3 < 128] = 1
b3[msk3 > 128] = 0

In [5]:
def metrics(b1, b2):
    intersection = np.sum(b1*b2)
    union = np.sum(b1) + np.sum(b2) - intersection

    iou = (intersection+1)/(union+1)
    # iou = (intersection)/(union)
    dice = (2*intersection+1)/(union+intersection+1)
    pixel_accuracy = np.sum(b1 == b2)/(512*512)

#     print("iou:{0}, dice:{1}, pixel:{2}".format(iou, dice, pixel_accuracy))
    return pixel_accuracy, iou, dice
    
metrics(b1, b3)

(0.92523193359375, 0.6932899349023536, 0.8188656926076872)

In [ ]:
import pandas as pd

results_matlab = pd.DataFrame(columns=["Image", "pixel", "iou", "dice"])
results_ddip = pd.DataFrame(columns=["Image", "pixel", "iou", "dice"])

for img_name in range(48):
    msk1 = skimage.io.imread('data/FISH/raw_annotations/{:04d}.png'.format(img_name))
    b1 = np.zeros(msk1.shape) # human label
    b1[msk1 > 0] = 1

    msk2 = skimage.io.imread('data/FISH/raw_masks/{:04d}.png'.format(img_name))
    b2 = np.zeros(msk2.shape)  # matlab
    b2[msk2 > 0] = 1

    msk3 = skimage.io.imread('output/FISH/final/{:04d}.png_mask.jpg'.format(img_name))
    b3 = np.zeros(msk3.shape) # ddip
    b3[msk3 < 128] = 1
    b3[msk3 > 128] = 0

    msk4 = skimage.io.imread('data/FISH/pred_masks/{:04d}.png'.format(img_name))
    b4 = np.zeros(msk4.shape) # my pred as gt
    b4[msk4 > 0] = 1
    
    p1, i1, d1 = metrics(b4, b2)
    res1 = {"Image": '{:04d}'.format(img_name), "pixel": p1, "iou": i1, "dice": d1}
    row1 = len(results_matlab)
    results_matlab.loc[row1] = res1
    
    p2, i2, d2 = metrics(b4, b3)
    res2 = {"Image": '{:04d}'.format(img_name), "pixel": p2, "iou": i2, "dice": d2}
    row2 = len(results_ddip)
    results_ddip.loc[row2] = res2
    
    plot_two(b2, b3, img_name)

In [58]:
results_matlab.mean().round(3)

Image    2.083750e+182
pixel     9.400000e-01
iou       7.610000e-01
dice      8.620000e-01
dtype: float64

In [24]:
results_matlab.mean().round(3)

Image    2.083750e+182
pixel     9.780000e-01
iou       8.880000e-01
dice      9.410000e-01
dtype: float64

In [25]:
results_ddip.mean().round(3)

Image    2.083750e+182
pixel     9.680000e-01
iou       8.410000e-01
dice      9.130000e-01
dtype: float64